In [2]:
import csv
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

get_query_args = FunctionDeclaration(
    name="query_to_csv",
    description="Get the csv columns from the query",
    parameters={
    "type": "object",
    "properties": {
        "company": {
            "type": "string",
            "description": "string of company name include ['Amazon', 'AMD', 'Amkor', 'Apple', 'Applied Material', 'Baidu', 'Broadcom', 'Cirrus Logic', 'Google', 'Himax', 'Intel', 'KLA', 'Marvell', 'Microchip', 'Microsoft', 'Nvidia', 'ON Semi', 'Qorvo', 'Qualcomm', 'Samsung', 'STM', 'Tencent', 'Texas Instruments', 'TSMC', 'Western Digital'],\
                如果使用者輸入的是中文，請對應以下的公司名稱；[亞馬遜', '超微', '艾克爾國際科技', '蘋果', '應用材料', '百度', '博通', '思睿邏輯', '谷歌', '奇景光電', '英特爾', '科磊', '邁威爾科技', '微芯科技', '微軟', '輝達', '安森美', '威訊聯合半導體', '高通公司', '三星', '意法半導體', '騰訊', '德州儀器', '台灣積體電路製造', '威騰電子']. \
                If user assign other company name, return out of data"
        },
        "index": {
            "type": "string",
            "description": "index arguement means the finacial index that you want to extract from the raw data frame\
                string of index include ['Cost of Goods Sold', 'Operating Expense', 'Operating Income', 'Revenue', 'Tax Expense', 'Total Asset' , 'Gross profit margin' , 'Operating margin']. \
                User can input more than one index. For example, user_query = '我想知道 Apple 在 2023 的 revenue 和 Cost of Goods Sold', you should return like this format ['Revenue' , 'Cost of Goods Sold']\
                So if user give multiple indeces input, return the list of indeces. \
                If user assign other index name, return out of data. \
                如果使用者輸入的是中文，請對應以下的指標名稱 ,['銷貨成本', '營業費用', '營業收入', '營收', '稅費', '總資產', '毛利率', '營業利益率']"
        },
        "year": {
            "type": "string",
            "description": "user can assign the year, from 2020 to 2024. If user assign 'all' or not assign, return '2020'."
        },
        "quarter": {
            "type": "string",
            "description": "user can assign the quarter, including ['Q1', 'Q2', 'Q3', 'Q4'], if user assign 'all' or not assign, return 'Q1'."
        },
    },
        "required": [
            "company",
            "index",
            "year",
            "quarter"
        ]
  }
)


In [ ]:
import vertexai

from vertexai.generative_models import GenerationConfig, GenerativeModel
PROJECT_ID = "tsmccareerhack2025-bsid-grp6"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "topic": {"type": "STRING"},
            "critical_point": {"type": "ARRAY", "items": {"type": "STRING"}},
        },
        "required": ["topic", "critical_point", "critical_data"],
    },
}

prompt = """
Summerize
    Apple Inc. (NASDAQ:AAPL) Q3 2024 Earnings Conference Call August 1, 2024 5:00 PM ET Company Participants Suhasini Chandramouli - Director of IR Tim Cook - CEO Luca Maestri - CFO Conference Call Participants Erik Woodring - Morgan Stanley Ben Reitzes - Melius Mike Ng - Goldman Sachs Amit Daryanani - Evercore Wamsi Mohan - Bank of America Krish Sankar - TD Cowen David Vogt - UBS Atif Malik - Citi Samik Chatterjee - JPMorgan Richard Kramer - Arete Research Suhasini Chandramouli Good afternoon and welcome to the Apple Q3 Fiscal Year 2024 Earnings Conference Call. My name is Suhasini Chandramouli, Director of Investor Relations. Today's call is being recorded. Speaking first today is Apple’s CEO, Tim Cook, and he'll be followed by CFO, Luca Maestri. After that, we'll open the call to questions from analysts. Please note that some of the information you'll hear during our discussion today will consist of forward-looking statements, including, without limitation, those regarding revenue, gross margin, operating expenses, other income and expense, taxes, capital allocation, and future business outlook including the potential impact of macroeconomic conditions on the company's business and results of operations. These statements involve risks and uncertainties that may cause actual results or trends to differ materially from our forecast. For more information, please refer to the risk factors discussed in Apple's most recently filed Annual Report on Form 10-K and the Form 8-K filed with the SEC today along with the associated press release. Apple assumes no obligation to update any forward-looking statements which speak only as of the date they are made. I'd now like to turn the call over to Tim for introductory remarks. Tim Cook  Thank you Suhasini. Good afternoon everyone and thanks for joining the call. Today, Apple is reporting a new June quarter revenue record of $85.8 billion, up 5% from a year ago and better than we had expected. EPS grew double digits to $1.40 and achieved a record for the June quarter. We also set quarterly revenue records in more than two dozen countries and regions, including Canada, Mexico, France, Germany, the UK, India, Indonesia, the Philippines
"""
from langchain_google_vertexai import VertexAI

# To use model
model = VertexAI(model_name="gemini-1.5-pro")
response = model.client.generate_content(
    prompt,
    generation_config=GenerationConfig(
        response_mime_type="application/json", response_schema=response_schema
    ),
)


In [15]:
response.text

'[{"topic": "Apple\'s Q3 2024 Earnings Report", "critical_point": ["Apple exceeded revenue expectations with a new June quarter record.", "The company achieved double-digit EPS growth and record EPS for the June quarter.", "Quarterly revenue records were set in numerous countries and regions globally." ], "critical_data": ["$85.8 billion in revenue, a 5% year-over-year increase.", "EPS of $1.40.", "Record revenue in over two dozen countries including Canada, Mexico, France, Germany, UK, India, Indonesia, and the Philippines." ]}]'

In [17]:
import json
analysis = json.loads(response.text)

In [18]:
analysis

[{'topic': "Apple's Q3 2024 Earnings Report",
  'critical_point': ['Apple exceeded revenue expectations with a new June quarter record.',
   'The company achieved double-digit EPS growth and record EPS for the June quarter.',
   'Quarterly revenue records were set in numerous countries and regions globally.'],
  'critical_data': ['$85.8 billion in revenue, a 5% year-over-year increase.',
   'EPS of $1.40.',
   'Record revenue in over two dozen countries including Canada, Mexico, France, Germany, UK, India, Indonesia, and the Philippines.']}]

In [ ]:
from langchain_google_vertexai import VertexAI

# To use model
model = VertexAI(model_name="gemini-1.5-pro")

'[{"recipe_name": "Chocolate Chip Cookies", "ingredients": ["1 cup (2 sticks) unsalted butter, softened", "1 cup granulated sugar", "1 cup packed brown sugar", "2 teaspoons pure vanilla extract", "2 large eggs", "3 cups all-purpose flour", "1 teaspoon baking soda", "1 teaspoon salt", "2 cups chocolate chips"]}, {"recipe_name": "Peanut Butter Cookies", "ingredients": ["1 cup creamy peanut butter", "1 cup granulated sugar", "1 cup packed brown sugar", "1 large egg", "1 teaspoon vanilla extract", "1 teaspoon baking soda"]}, {"recipe_name": "Oatmeal Raisin Cookies", "ingredients": ["1 cup (2 sticks) unsalted butter, softened", "1 cup granulated sugar", "1 cup packed brown sugar", "2 large eggs", "1 teaspoon vanilla extract", "1 1/2 cups all-purpose flour", "1 teaspoon baking soda", "1 teaspoon ground cinnamon", "1/2 teaspoon salt", "3 cups rolled oats", "1 cup raisins"]}]'

In [26]:
model = GenerativeModel("gemini-1.5-pro")

plot_tool = Tool(
    function_declarations=[get_query_args],
)

prompt = """Apple 在 2021 年的 Total Asset（總資產）是多少？"""

response = model.generate_content(
    prompt,
    tools=[plot_tool],
)

In [27]:
response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "query_to_csv"
        args {
          fields {
            key: "year"
            value {
              string_value: "2021"
            }
          }
          fields {
            key: "quarter"
            value {
              string_value: "all"
            }
          }
          fields {
            key: "index"
            value {
              list_value {
                values {
                  string_value: "Total Asset"
                }
              }
            }
          }
          fields {
            key: "company"
            value {
              string_value: "Apple"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.142578125
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.103515625
  }
  safety_ratings {
    category:

In [28]:
search_params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    search_params[key] = value
search_params

{'year': '2021',
 'company': 'Apple',
 'index': ['Total Asset'],
 'quarter': 'all'}

In [29]:
import pandas as pd

def get_csv_data(search_params):
    df = pd.read_csv("FIN_Data.csv")
    # Filter data
    filtered_df = df[
        (df['Company Name'] == search_params['company']) &
        (df['Index'] == search_params['index']) &
        (df['CALENDAR_YEAR'].astype(str) == search_params['year']) &
        (df['CALENDAR_QTR'] == search_params['quarter'])
    ]
    if not filtered_df.empty:
        usd_value = filtered_df['USD_Value'].values[0]
        print(f"USD Value: {usd_value}")
        return str(usd_value)
    else:
        return "No matching record found."
    
self_function_response = get_csv_data(search_params)
self_function_response

ValueError: ('Lengths must match to compare', (2832,), (1,))

In [24]:
response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Give your answer concise and clear."""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "get_query_args",
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="get_query_args",
            response={
                "content": self_function_response,
            }
        )
    ]),
    ],
    tools=[plot_tool],
)


response.candidates[0].content.parts[0].text

"I cannot answer this question. The available tool does not have the information about Apple's Total Asset. \n"